In [3]:
# !gdown 1-S_DPVzrTNdzyRqb5hSrsmLk2c0-6tpk

Downloading...
From: https://drive.google.com/uc?id=1-S_DPVzrTNdzyRqb5hSrsmLk2c0-6tpk
To: /content/robert_frost.txt
100% 56.3k/56.3k [00:00<00:00, 55.6MB/s]


In [3]:
import numpy as np
import string


# P(w1) - Probability distribution for first words in lines
# initial_probabilities[word] = count(word as first) / total_lines
initial_probabilities = {}
# P(wt|wt-1) - First-order transition probabilities
# For word given one previous word
# first_order_transitions[prev_word][curr_word] = count(prev_word, curr_word) / count(prev_word)
first_order_transitions = {}
# P(wt|wt-1,wt-2) - Second-order transition probabilities
# For word given two previous words
# second_order_transitions[(prev_prev_word, prev_word)][curr_word] =
#     count(prev_prev_word, prev_word, curr_word) / count(prev_prev_word, prev_word)
second_order_transitions = {}

def remove_punctuation(text):
    """
    Removes all punctuation from input string using string.punctuation
    Args:
        text (str): Input string with possible punctuation
    Returns:
        str: Clean string without punctuation
    """
    return text.translate(str.maketrans('', '', string.punctuation))


def add_to_transition_dict(transition_dict, condition, next_word):
    """
    Helper function to build frequency lists in transition dictionaries

    Args:
        transition_dict (dict): Target transition dictionary to modify
        condition: Condition state (word or tuple of words)
        next_word: Following word to record
    """
    if condition not in transition_dict:
        transition_dict[condition] = []
    transition_dict[condition].append(next_word)

with open(
    r'C:\Users\lilit\PycharmProjects\nlp-course-2025.1\Lilit Mnatsakanyan\data\tumanyan.txt',
    'r',
    encoding='utf-8',
    errors='replace'
) as file:
    text = file.read()

# Training phase - Process input text file line by line
for line in text.split('\n'):
    # Clean the line: remove punctuation, convert to lowercase, and split into words
    clean_line = remove_punctuation(line.strip().lower())
    words = clean_line.split()
    line_length = len(words)

    # Process each word in the line
    for position in range(line_length):
        current_word = words[position]

        if position == 0:
            # First word in line - update initial word frequencies
            # Formula: P(w1) = count(w1 as first word) / total number of lines
            initial_probabilities[current_word] = initial_probabilities.get(current_word, 0.) + 1
        else:
            previous_word = words[position - 1]

            if position == line_length - 1:
                # Last word in line - mark as possible ending
                add_to_transition_dict(second_order_transitions,
                                       (previous_word, current_word), 'END')

            if position == 1:
                # Second word in line - update first-order transitions
                # Formula: P(wt|wt-1) = count(wt-1, wt) / count(wt-1)
                add_to_transition_dict(first_order_transitions, previous_word, current_word)
            else:
                # All other words - update second-order transitions
                # Formula: P(wt|wt-2,wt-1) = count(wt-2,wt-1,wt) / count(wt-2,wt-1)
                previous_previous_word = words[position - 2]
                add_to_transition_dict(second_order_transitions,
                                       (previous_previous_word, previous_word),
                                       current_word)

# Normalize initial probabilities
# P(w1) = count(w1 as first word) / total number of lines
total_lines = sum(initial_probabilities.values())
for word, count in initial_probabilities.items():
    initial_probabilities[word] = count / total_lines


def convert_to_probability_dict(word_list):
    """
    Converts a list of words into a probability distribution dictionary

    Formula: P(word) = count(word) / total_words

    Args:
        word_list (list): List of words
    Returns:
        dict: Dictionary mapping words to their probabilities
    Example:
        Input: ['cat', 'dog', 'cat']
        Output: {'cat': 0.67, 'dog': 0.33}
    """
    probability_dict = {}
    total_words = len(word_list)

    # Count frequencies
    for word in word_list:
        probability_dict[word] = probability_dict.get(word, 0.) + 1

    # Convert to probabilities
    for word, count in probability_dict.items():
        probability_dict[word] = count / total_words

    return probability_dict


# Convert frequency lists to probability distributions
for condition_word, next_words in first_order_transitions.items():
    first_order_transitions[condition_word] = convert_to_probability_dict(next_words)

for condition_pair, next_words in second_order_transitions.items():
    second_order_transitions[condition_pair] = convert_to_probability_dict(next_words)


def sample_from_distribution(probability_dict):
    """
    CUMULATIVE PROBABILITY SAMPLING METHOD EXPLANATION

    Given a discrete probability distribution:
    words = {'cat': 0.3, 'dog': 0.5, 'bird': 0.2}

    The cumulative probabilities would be:
    'cat':  0.0 to 0.3  (0.3)
    'dog':  0.3 to 0.8  (0.3 + 0.5)
    'bird': 0.8 to 1.0  (0.3 + 0.5 + 0.2)

    When we generate a random number between 0 and 1:
    - If random = 0.2 → returns 'cat' (falls in 0.0-0.3)
    - If random = 0.6 → returns 'dog' (falls in 0.3-0.8)
    - If random = 0.9 → returns 'bird' (falls in 0.8-1.0)

///////////////////////////////////////////////////////////////////////////////////

    VISUAL EXPLANATION OF CUMULATIVE PROBABILITY SAMPLING

    1. Original Probabilities:
    cat  = 0.3  (30% chance)
    dog  = 0.5  (50% chance)
    bird = 0.2  (20% chance)

    2. We create a "number line" from 0 to 1 and divide it into segments:

    0.0 -------- 0.3 -------------- 0.8 -------- 1.0
         cat          dog              bird
    |----30%-----|------50%------|----20%-----|

    3. Random number generation acts like throwing a dart at this line:

    Example 1: Random number = 0.2
    0.0 -------- 0.3 -------------- 0.8 -------- 1.0
           ↑
          0.2
        (picks cat)

    Example 2: Random number = 0.6
    0.0 -------- 0.3 -------------- 0.8 -------- 1.0
                        ↑
                       0.6
                     (picks dog)

    Example 3: Random number = 0.9
    0.0 -------- 0.3 -------------- 0.8 -------- 1.0
                                         ↑
                                        0.9
                                     (picks bird)

///////////////////////////////////////////////////////////////////////////////////

    EXAMPLE OF WHY WE NEED PROBABILITY SAMPLING IN POETRY GENERATION

    Let's say we're analyzing this simple poem:
    'the cat sat'
    'the cat napped'
    'the dog sat'

    After analysis, our Markov chain would have these probabilities:

    1. First word probabilities (initial_probabilities):
       'the': 1.0  (appears 3/3 times as first word)

    2. First-order transitions (after seeing 'the'):
       first_order_transitions['the'] = {
           'cat': 0.67,  # 'cat' appears 2/3 times after 'the'
           'dog': 0.33   # 'dog' appears 1/3 times after 'the'
       }

    3. Second-order transitions (after seeing 'the cat'):
       second_order_transitions[('the', 'cat')] = {
           'sat': 0.5,    # After 'the cat', 'sat' appears 1/2 times
           'napped': 0.5  # After 'the cat', 'napped' appears 1/2 times
       }

////////////////////////////////////////////////////////////////////////////////////

    Randomly samples a word from a probability distribution
    Uses cumulative probability method for sampling

    Formula: Find word where cumsum(P(words)) > random(0,1)

    Args:
        probability_dict (dict): Dictionary of word probabilities
    Returns:
        str: Sampled word
    """
    random_value = np.random.random()
    cumulative_probability = 0.

    for word, probability in probability_dict.items():
        cumulative_probability += probability
        if random_value < cumulative_probability:
            return word


def select_most_frequent_word(probability_dict):
    """
    Selects the word that appears most frequently in the dictionary.

    Example:
    If we have probability_dict = {'cat': 0.3, 'dog': 0.5, 'bird': 0.2}
    This means:
        - 'cat' appears 30% of the time
        - 'dog' appears 50% of the time
        - 'bird' appears 20% of the time
    The function will return 'dog' because 0.5 (50%) is the highest probability

    Args:
        probability_dict (dict): Dictionary where:
            - keys are words
            - values are their probabilities
    Returns:
        str: The word with highest probability
    """
    # Initialize variables to keep track of the most frequent word
    highest_probability = 0.0
    most_frequent_word = None

    # Go through each word and its probability
    for word, probability in probability_dict.items():
        # If we find a word with higher probability than our current highest
        if probability > highest_probability:
            # Update our tracking variables
            highest_probability = probability
            most_frequent_word = word

    return most_frequent_word


def generate_poetry():
    """
    Generates 4 lines of poetry using the trained Markov model

    Generation process uses these probability distributions:
    1. First word: P(w1)
    2. Second word: P(w2|w1)
    3. Subsequent words: P(wt|wt-2,wt-1)
    """
    for _ in range(10):  # Generate 4 lines
        line_words = []

        # Sample first word from initial distribution
        first_word = sample_from_distribution(initial_probabilities)
        # first_word = select_most_frequent_word(initial_probabilities)
        line_words.append(first_word)

        # Sample second word based on first word
        second_word = sample_from_distribution(first_order_transitions[first_word])
        # second_word = select_most_frequent_word(first_order_transitions[first_word])
        line_words.append(second_word)

        # Generate rest of the line
        while True:
            # Get next word based on previous two words
            previous_previous = first_word
            previous_word = second_word

            next_word = sample_from_distribution(second_order_transitions[(previous_previous, previous_word)])
            # next_word = select_most_frequent_word(second_order_transitions[(previous_previous, previous_word)])

            if next_word == 'END':
                break

            line_words.append(next_word)

            # Shift window of words
            first_word = previous_word
            second_word = next_word

        print(' '.join(line_words))


# Generate poetry
generate_poetry()

սևուկ ուլիկ
«բարի օր»— ասաց աղքատն ու պատմեց սոված գայլի սիրուն աղջկա ու չոր ծառի ապսպրանքը։
գիշերը դարձյալ դերվիշի շոր մտավ հարուն ալ ռաշիդ թագավորը։ հարուն ալ ռաշիդ թագավորը սովորություն ուներ՝ շորերը փոխած ման էր գալիս իմանալու թե ինչ պատասխան տանք։ հիմի մեզ ինչ անի մենք ենք մեղավոր վայ թե քեզ հետ կենամ բաժանվում եմ իմ բաժինը տուր գնամ ջոկ ապրեմ։
— որը կպատահի։
էս վախկոտ նազարը մի անշնորհք ու ալարկոտ մարդ է պատահում։ էս ծեր մարդը մի քանի հասցրի ինչ ուներ՝ առաջիս փռեց։ իմ տասը մանեթը տուր մնացածը քու փողն է ընչի՞ս է պետք
— վա՜յ— ասում է— էս էլ քո սիրած քույրը տանը ինչ ունեինքչունեինք՝ ջարդեց։
մարդը կնկանն է ասում հիմար կնիկը մարդուն ու միշտ կռվելիս են լինում։ ախպերը պսակվում է կնիկ է բերում։
գալիս են ի՞նչ ես միտք անում— հարցնում է թագավորը։
լուսադեմին աքլորը կանչեց՝ ծուղրուղո՜ւ։
— ուրեմն արծիվը ճուտ է եղել։
